### Modelos baseados em árvores

<br>

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'darkgrid')

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [3]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.299404995551587  /  0.03885969324195987
estimators:  10  prof:  5  | R2 mean / std:  0.4110591572429194  /  0.05918721014482146
estimators:  10  prof:  10  | R2 mean / std:  0.36109220319373136  /  0.05959708769970797
estimators:  100  prof:  1  | R2 mean / std:  0.3423582683734492  /  0.02859521546494546
estimators:  100  prof:  5  | R2 mean / std:  0.4253338502713404  /  0.04449964775479308
estimators:  100  prof:  10  | R2 mean / std:  0.42575630139451653  /  0.024636520501367203
estimators:  100  prof:  1  | R2 mean / std:  0.3246461930584648  /  0.04398125243811048
estimators:  100  prof:  5  | R2 mean / std:  0.4329811913085015  /  0.038290494429131024
estimators:  100  prof:  10  | R2 mean / std:  0.4340969904136609  /  0.03436702580055987


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [5]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [6]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [7]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3, n_jobs = -1)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [8]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [9]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [10]:
grid.best_score_

0.42528109564360195

In [11]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [12]:
grid.cv_results_

{'mean_fit_time': array([0.02943055, 2.002623  , 0.03143676, 2.73169986]),
 'std_fit_time': array([0.00170532, 0.03173744, 0.00047233, 0.06453727]),
 'mean_score_time': array([0.00301067, 0.11437734, 0.00301067, 0.09631824]),
 'std_score_time': array([6.25769923e-07, 1.20679195e-02, 8.10467325e-07, 2.98084054e-02]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.38124835, 0.34849456, 0.3366896 , 0.37446907]),
 'split1_test_score': array([0.44139622, 0.44784718, 0.42905037, 0.45058935]),
 'split2_test_score': array(

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [13]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [14]:
class gridSearchAll():
    
    def __init__(self):
        self.grid_models = []
        #scoring...
        #num_folds...
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
        
    #def fit_all(...)
    #    ...
    
    #def best_all_grid_models(...)
    #    ...

In [15]:
gd = gridSearchAll()

In [16]:
gd.grid_models

[]

In [17]:
params_RF

{'n_estimators': [10, 1000], 'max_depth': [2, 10]}

In [18]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [19]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}]]

In [20]:
from sklearn.neighbors import KNeighborsRegressor

In [21]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [22]:
gd.grid_models

[[RandomForestRegressor(), {'n_estimators': [10, 1000], 'max_depth': [2, 10]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}]]

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

### Exercício 1

In [23]:
df.head()

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


Antes de aplicar qualquer modelo de Machine Learning nos nossos dados, precisamos fazer um pré-processamento, seguindo os seguintes passos:

__1 - Verificar dados faltantes__

__2 - Fazer engenharia de dados__

__3 - Transfomar a variável 'proximidade_ao_mar' em 0 ou 1 caso seja binaria, ou fazer o One Hot encoding, caso contrario.__

__4 - Normalizar as variáveis quantitativas.__

A duas primeiras partes faremos antes da divisão dos dados em treino e teste, o OneHotEnconding e a normalização, faremos depois da divisão



### 1 - Dados faltantes

In [24]:
# conferindo o tamanho total do DataSet

df.shape

(20640, 10)

In [25]:
# conferindo os dados nulos

df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [26]:
df['total_quartos'].isnull().sum()/df.shape[0]

0.01002906976744186

Em termos de porcentagem, a quantidade de registros 'total_quartos' é bem pequena, vamos usar o describe para ver os dados da coluna

In [27]:
df['total_quartos'].describe()

count    20433.000000
mean       537.870553
std        421.385070
min          1.000000
25%        296.000000
50%        435.000000
75%        647.000000
max       6445.000000
Name: total_quartos, dtype: float64

Podemos ver que a média e mediana estão bem próximos,como o total de quartos é sempre um número natural, sem casas decimais, vamos trocar os dados nulos do nosso DataSet pela mediada.

In [28]:
mediana_total_quartos = df['total_quartos'].median()

In [29]:
df['total_quartos'] = df['total_quartos'].fillna(mediana_total_quartos)

In [30]:
df.isnull().sum()

longitude                  0
latitude                   0
idade_mediana_das_casas    0
total_comodos              0
total_quartos              0
populacao                  0
familias                   0
salario_mediano            0
preco_mediano_das_casas    0
proximidade_ao_mar         0
dtype: int64

__Engenharia de Dados:__

Vamos criar mais variáveis para nosso dataset, são elas:

__Média do total de quartos por familia__

__Tamanho médio por familia__

In [31]:
df['quartos_por_familia'] = df['total_quartos']/df['familias']
df['populacao_por_familia']= df['populacao']/df['familias']

In [32]:
df.head()

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar,quartos_por_familia,populacao_por_familia
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA,1.023810,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA,0.971880,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA,1.073446,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA,1.073059,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA,1.081081,2.181467


__Tratamento da variável Proximidade ao Mar e normalização das outras variáveis__

Como podemos ver, essa é a nossa única variável categórica no dataset, portanto vamos inicialmente analisar seus dados.

In [33]:
df['proximidade_ao_mar'].value_counts()

<1H OCEANO       9136
INTERIOR         6551
PERTO OCEANO     2658
PERTO DA BAÍA    2290
ILHA                5
Name: proximidade_ao_mar, dtype: int64

Temos então que nossa variável não é binária, logo vamos fazer o pré-processamento nela usando o OneHotEncoding, como não temos mais variáveis categóricas, iremos normalizar todas as outras, antes disso, vamos dividir nossos dados em treino e teste

In [34]:
dftreino, dfteste = train_test_split(df, test_size = 0.25, random_state = 0)

In [35]:
dftreino

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar,quartos_por_familia,populacao_por_familia
19226,-122.68,38.46,19.0,4976.0,711.0,1926.0,625.0,7.3003,381300.0,<1H OCEANO,1.137600,3.081600
14549,-117.24,32.95,18.0,1591.0,268.0,547.0,243.0,5.9547,329300.0,PERTO OCEANO,1.102881,2.251029
9093,-118.27,34.68,19.0,552.0,129.0,314.0,106.0,3.2125,185400.0,INTERIOR,1.216981,2.962264
12213,-117.18,33.51,13.0,270.0,42.0,120.0,42.0,6.9930,500001.0,<1H OCEANO,1.000000,2.857143
12765,-121.41,38.62,21.0,3260.0,763.0,1735.0,736.0,2.5162,97500.0,INTERIOR,1.036685,2.357337
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,-121.26,38.27,20.0,1314.0,229.0,712.0,219.0,4.4125,144600.0,INTERIOR,1.045662,3.251142
19648,-120.89,37.48,27.0,1118.0,195.0,647.0,209.0,2.9135,159400.0,INTERIOR,0.933014,3.095694
9845,-121.90,36.58,31.0,1431.0,435.0,704.0,393.0,3.1977,289300.0,PERTO OCEANO,1.106870,1.791349
10799,-117.93,33.62,34.0,2125.0,498.0,1052.0,468.0,5.6315,484600.0,<1H OCEANO,1.064103,2.247863


In [36]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [37]:
def preprocessamento_completo(df, dataset_de_treino = True, cat_encoder = None, std_scaler = None):

    dff = df.copy()
    
    variaveis_para_normalizar = ['latitude',
                                 'longitude',
                                'idade_mediana_das_casas',
                                 'total_comodos',
                                 'total_quartos',
                                 'populacao',
                                 'familias',
                                 'salario_mediano',
                                 'quartos_por_familia',
                                 'populacao_por_familia']

    if dataset_de_treino:  
        
        encoder = OneHotEncoder()
        df_prox_mar_OHE = encoder.fit_transform(dff[['proximidade_ao_mar']]).toarray()

        sc = StandardScaler()
        variaveis_norm = sc.fit_transform(dff[variaveis_para_normalizar])
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y, encoder, sc
    
    else:
        df_prox_mar_OHE = cat_encoder.transform(dff[['proximidade_ao_mar']]).toarray()
        
        variaveis_norm = std_scaler.transform(dff[variaveis_para_normalizar]) 
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y

In [38]:
Xtreino, ytreino, encod, stand = preprocessamento_completo(df = dftreino, dataset_de_treino=True)

In [39]:
Xtreino.shape, ytreino.shape

((15480, 15), (15480,))

### Temos então nossos dados de treino, vamos usar o OHE e Standard Scaller já fitados neles, e criar nosso dataset de Teste

In [40]:
Xteste, yteste = preprocessamento_completo(df = dfteste, dataset_de_treino=False, cat_encoder=encod, std_scaler=stand)

In [41]:
Xteste.shape, yteste.shape

((5160, 15), (5160,))

## Enfim finalizamos o pré-processamento, e podemos então partir para o Machine Learning

Vamos criar uma lista com estimadores que queremos usar, isto é, o KNN's, Random forests e GradientBoosting

In [42]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [43]:
estimadores = [RandomForestRegressor(), KNeighborsRegressor(), GradientBoostingRegressor()]

# criaremos 3 dicionários, um com a variação de parâmetros para cada estimador.

params_RF = {'n_estimators':[10,50,100,500,1000], 'max_depth':[2,5,10]}
params_KNN = {'n_neighbors': [2,10,20,50]}
params_GB = {'n_estimators': [10,100,200,1000], 'max_depth': [3,5,7,10]}

### Vamos fazer os grids individualmente para cada estimador.

__1 - Random Forest:__

In [44]:
import time

In [45]:
gridRF = GridSearchCV(estimator = estimadores[0], 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 5, n_jobs = -1)

gridRF

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 10],
                         'n_estimators': [10, 50, 100, 500, 1000]},
             scoring='r2')

In [46]:
gridRF.fit(Xtreino,ytreino)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 10],
                         'n_estimators': [10, 50, 100, 500, 1000]},
             scoring='r2')

__Verificando os melhores parâmetros e Score obtido__

In [47]:
gridRF.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [48]:
gridRF.best_score_

0.785396797731448

__2 - KNeighbors:__


In [49]:
gridKNN = GridSearchCV(estimator = estimadores[1], 
                    param_grid = params_KNN, 
                    scoring = 'r2', 
                    cv = 5)

gridKNN

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 10, 20, 50]}, scoring='r2')

In [50]:
gridKNN.fit(Xtreino, ytreino)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 10, 20, 50]}, scoring='r2')

__Verificando melhores parâmetros e Score obtido__

In [51]:
gridKNN.best_params_

{'n_neighbors': 10}

In [52]:
gridKNN.best_score_

0.7266521342787959

__3 - GradientBoosting:__

In [53]:
gridGB = GridSearchCV(estimator = estimadores[2], 
                    param_grid = params_GB, 
                    scoring = 'r2', 
                    cv = 5, n_jobs=-1)

gridGB

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10],
                         'n_estimators': [10, 100, 200, 1000]},
             scoring='r2')

In [54]:
gridGB.fit(Xtreino, ytreino)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10],
                         'n_estimators': [10, 100, 200, 1000]},
             scoring='r2')

Verificando melhores parâmetros e Score obtido:

In [55]:
gridGB.best_params_

{'max_depth': 5, 'n_estimators': 1000}

In [56]:
gridGB.best_score_

0.8320699365894997

### Como já era esperado, o ensemble Gradient Boosting tornou o melhor resultado com o Melhor R2 score.

Finalizamos então o primeiro exercício.

Exercício 2: Crie uma classe para comparar o grid_search dentre vários modelos distintos.

Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de completar essa classe. Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe. Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [57]:
class gridSearchAll():
    
    def __init__(self):
        self.grid_models = []
        #scoring...
        #num_folds...
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
        
    #def fit_all(...)
    #    ...
    
    #def best_all_grid_models(...)
    #    ...

### Vamos trabalhar por partes e no fim construir o modelo completo.

Inicialmente queremos que no método construtor tenha:

1 - Os modelos com os parâmetros que o GridSeach vai usar.

2 - A métrica de perfomance a ser avaliada

3 - A Quantidade de Folds para a Validaçao cruzada.

Além disso, colocaremos uma variável para salvar os melhor grid de cada estimador

In [58]:
class gridSearchAll():
    
    def __init__(self, modelos, metrica, folds):
        self.grid_models = modelos
        self.grid.metrica = metrica
        self.num_folds = folds
        self.best_grids = []

Assim ao construir criar um objeto da classe, definiremos os modelos que queremos avaliar, já com os parâmetros, métrica de perfomance, agora adicionaremos o método já criado no exemplo, que serve para adicionar outros estimadores para serem avaliados.

In [59]:
class gridSearchAll():
    
    def __init__(self, modelos, metrica, folds):
        
        self.grid_models = modelos
        self.grid.metrica = metrica
        self.num_folds = folds
        self.best_grids = []
    
    def insert_model(self, estimator_base, param_grid):
        
        self.grid_models.append([estimator_base, param_grid])


### Próximo passo é adicionar um método que vai fitar todos os modelos adicionados no objeto através o GridSeach e adicionar o Grid treinado no lista "best_grids"

In [60]:
class gridSearchAll():
    
    def __init__(self, modelos, metrica, folds):
        
        self.grid_models = modelos
        self.grid.metrica = metrica
        self.num_folds = folds
        self.best_grids = []
    
    def insert_model(self, estimator_base, param_grid):
        
        self.grid_models.append([estimator_base, param_grid])
    
    def fit_all(X,y):
        for est in self.grid_models:
            grid = GridSearchCV(estimator = self.est[0], 
                    param_grid = self.est[1], 
                    scoring = metrica, 
                    cv = folds)
            grid.fit(X,y)
            self.best_grids.append(grid)

### Por fim, colocaremos o método que nos mostra o melhor de todos os modelos

In [61]:
class gridSearchAll():
    
    def __init__(self, modelos, metrica, folds):
        
        self.grid_models = modelos
        self.grid_metrica = metrica
        self.num_folds = folds
        self.best_grids = []
    
    def insert_model(self, estimator_base, param_grid):
        
        self.grid_models.append([estimator_base, param_grid])
    
    def fit_all(self, X, y):
        for est in self.grid_models:
            grid = GridSearchCV(estimator = est[0], 
                    param_grid = est[1], 
                    scoring = self.grid_metrica, 
                    cv = self.num_folds, n_jobs =-1)
            grid.fit(X,y)
            self.best_grids.append(grid)
    
    def best_all_grid_models(self):
        
        melhor_score = self.best_grids[0].best_score_
        melhor_grid = self.best_grids[0]
        for n in list(range(1,len(self.best_grids))):
            if self.best_grids[n].best_score_ > melhor_score:
                melhor_score = self.best_grids[n].best_score_
                melhor_grid = self.best_grids[n]
            else:
                continue
        return melhor_grid

### Terminamos então nossa classe, vamos então para o último exercício, onde aplicaremos os mesmos modelos do exercício 1 no nosso Dataset, porém usando a classe gridSeachAll.

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

Separando os modelos e parâmetros:

In [62]:
RF = RandomForestRegressor() 
KNN = KNeighborsRegressor() 
GB = GradientBoostingRegressor()

params_RF = {'n_estimators':[10,50,100,500,1000], 'max_depth':[2,5,10]}
params_KNN = {'n_neighbors': [2,10,20,50]}
params_GB = {'n_estimators': [10,100,200,1000], 'max_depth': [3,5,7,10]}

Modelo_RF = [RF, params_RF]
Modelo_KNN = [KNN, params_KNN]
Modelo_GB = [GB, params_GB]

Modelos = [Modelo_RF, Modelo_KNN, Modelo_GB]

Criando o objeto da classe:

In [63]:
gridall = gridSearchAll(Modelos, 'r2', 5)

In [ ]:
t0 = time.time()

# fitando com todos os modelos

gridall.fit_all(Xtreino, ytreino)

t1 = time.time()

print('Tempo de execução: ', (t1-t0)/60, 'minutos.')

### Conseguimos então retornar os melhores modelos de cada grid

In [ ]:
gridall.best_grids

In [ ]:
gridall.best_grids[0].best_params_

In [ ]:
gridall.best_grids[0].best_score_

In [ ]:
gridall.best_grids[1].best_params_

In [ ]:
gridall.best_grids[1].best_score_

In [ ]:
gridall.best_grids[2].best_params_

In [ ]:
gridall.best_grids[2].best_score_

### Temos os mesmos parâmetros que tinhamos obtido fazendo separadamente cada.

Poderiamos também saber o melhor dos modelos sem precisar comparar um por um, apenas usando o método best_all_grid_models.


In [ ]:
Melhor_modelo = gridall.best_all_grid_models()

In [ ]:
Melhor_modelo

In [ ]:
Melhor_modelo.best_score_

# Finalizamos aqui o notebook e o exercício do módulo.